In [11]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import re
import tensorflow as tf
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# test review 전처리

In [77]:
import csv

list = []
f = open('testset.csv','r', encoding = 'cp949')
rdr = csv.reader(f)
for line in rdr:
    print(line)
    list.append(line)
f.close()

['review']
['영상보자마자 쓰러질뻔했어요ㅜㅜ너무멋있어서 출구완전봉쇄 절대못나오는회전문!!!!!']
['뭔베짱으로지금 티비에나오냐ㅡㅡ 걍 짜져있어도모자랄판에']
['신성 잘했어요.']
['플랙스니 뭐니 돈자랑하는 힙합문화 솔직히 졸부들 천박한 문화같아. 싼티나. 애들이 보고 배울까 겁나네']
['너무너무 멋있어요']
['항상 응원해요']
['진짜 못생겼다 우웩']
['짜증난다 티비에 그만나와']
['멋있다!']
['정신분열 잼ㅋㅋㅋㅋ']


In [78]:
testset = pd.read_csv('testset.csv', encoding = 'cp949')

In [79]:
testset = testset[['review']]
testset

,review
0,영상보자마자 쓰러질뻔했어요ㅜㅜ너무멋있어서 출구완전봉쇄 절대못나오는회전문!!!!!
1,뭔베짱으로지금 티비에나오냐ㅡㅡ 걍 짜져있어도모자랄판에
2,신성 잘했어요.
3,플랙스니 뭐니 돈자랑하는 힙합문화 솔직히 졸부들 천박한 문화같아. 싼티나. 애들이 ...
4,너무너무 멋있어요
5,항상 응원해요
6,진짜 못생겼다 우웩
7,짜증난다 티비에 그만나와
8,멋있다!
9,정신분열 잼ㅋㅋㅋㅋ


In [80]:
data = testset['review']

In [81]:
normalized_text = []
for string in data.tolist():
    try:
        tokens = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣]+", " ", string.lower())
    except Exception as e:
        print(string)
        break
    normalized_text.append(tokens)
print(normalized_text)

['영상보자마자 쓰러질뻔했어요ㅜㅜ너무멋있어서 출구완전봉쇄 절대못나오는회전문 ', '뭔베짱으로지금 티비에나오냐ㅡㅡ 걍 짜져있어도모자랄판에', '신성 잘했어요 ', '플랙스니 뭐니 돈자랑하는 힙합문화 솔직히 졸부들 천박한 문화같아 싼티나 애들이 보고 배울까 겁나네', '너무너무 멋있어요', '항상 응원해요', '진짜 못생겼다 우웩', '짜증난다 티비에 그만나와', '멋있다 ', '정신분열 잼ㅋㅋㅋㅋ']


In [82]:
testset.review = normalized_text
for sentence in testset['review']:
    print(sentence)

영상보자마자 쓰러질뻔했어요ㅜㅜ너무멋있어서 출구완전봉쇄 절대못나오는회전문 
뭔베짱으로지금 티비에나오냐ㅡㅡ 걍 짜져있어도모자랄판에
신성 잘했어요 
플랙스니 뭐니 돈자랑하는 힙합문화 솔직히 졸부들 천박한 문화같아 싼티나 애들이 보고 배울까 겁나네
너무너무 멋있어요
항상 응원해요
진짜 못생겼다 우웩
짜증난다 티비에 그만나와
멋있다 
정신분열 잼ㅋㅋㅋㅋ


In [83]:
from konlpy.tag import Okt  
okt = Okt()

In [84]:
X_token=[]
for sentence in testset['review']:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    #temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_token.append(temp_X)

In [85]:
max_words = 35000
tokenizer = Tokenizer(num_words = max_words) # 상위 35,000개의 단어만 보존
tokenizer.fit_on_texts(X_token) 
X_token = tokenizer.texts_to_sequences(X_token)

In [86]:
word_to_index = tokenizer.word_index
vocab_size = len(word_to_index)+1

In [87]:
max_len = 124
X_data = pad_sequences(X_token, maxlen=max_len)

# model 불러오기

In [92]:
from tensorflow.keras.models import load_model
model = load_model('model11.h5')

In [93]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         3500000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 3,631,937
Trainable params: 3,631,937
Non-trainable params: 0
______________________________________________

In [94]:
predict = model.predict_classes(X_data)
for i in range(len(X_data)):
    if(predict[i] == 0):
        print("선플")
    else:
        print("악플")

선플
선플
악플
선플
악플
선플
악플
선플
악플
악플
